In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
# 读取视频文件
video_path = 'media/video_input_01.mp4'
cap = cv2.VideoCapture(video_path)

# 获取视频的总帧数和帧率
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))

print(f"Total Frames: {total_frames}")
print(f"Frame Rate: {frame_rate} fps")

Total Frames: 1190
Frame Rate: 60 fps


In [3]:
# 选择感兴趣区域（ROI）
print("Select a ROI and then press SPACE or ENTER button!")
print("Cancel the selection process by pressing c button!")

while True:
    ret, first_frame = cap.read()
    if not ret:
        break

    cv2.imshow("Select ROI", first_frame)
    roi = cv2.selectROI("Select ROI", first_frame, fromCenter=False)
    cv2.destroyWindow("Select ROI")

    if roi[2] > 0 and roi[3] > 0:  # 确保选择了有效的ROI
        break

# 获取感兴趣区域的坐标
x, y, w, h = map(int, roi)

Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


In [4]:
# 初始化追踪点（使用ROI的中心点）
point_of_interest = np.array([[x + w // 2, y + h // 2]], dtype=np.float32)
p0 = point_of_interest
p0

array([[914., 550.]], dtype=float32)

In [5]:
# 读取第一帧图像
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # 回到视频的第一帧
ret, first_frame = cap.read()
gray_first_frame = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)  # 将第一帧转换为灰度图像

# 创建一个空的轨迹列表
tracks = []

# 设置采样频率（在这里使用视频帧率）
sampling_rate = frame_rate

# 创建掩码
mask = np.zeros_like(first_frame)

In [6]:
try:
    frame_count = 0  # 用于计算帧数的变量

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 打印帧数信息
        frame_count += 1
        print(f"Processing Frame {frame_count} / {total_frames}")

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # 使用Canny边缘检测来提取边缘信息
        edges = cv2.Canny(gray_frame, 50, 150)

        # 获取感兴趣区域的内容
        roi_frame = edges[y:y+h, x:x+w]

        # 寻找边缘上的点，这里使用Hough Line Transform
        lines = cv2.HoughLinesP(roi_frame, 1, np.pi / 180, threshold=100, minLineLength=50, maxLineGap=10)

        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                # 将边缘点的坐标添加到追踪列表中
                tracks.append(np.array([[x1, y1]], dtype=np.float32))

                # 在图像中绘制特征点
                frame = cv2.circle(frame, (x1, y1), 5, (0, 0, 255), -1)

        # 显示结果
        cv2.imshow("Frame", frame)

        # 按ESC键退出
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break

except KeyboardInterrupt:
    pass

finally:
    # 关闭视频文件
    cap.release()
    cv2.destroyAllWindows()

Processing Frame 1 / 1190
Processing Frame 2 / 1190
Processing Frame 3 / 1190
Processing Frame 4 / 1190
Processing Frame 5 / 1190
Processing Frame 6 / 1190
Processing Frame 7 / 1190
Processing Frame 8 / 1190
Processing Frame 9 / 1190
Processing Frame 10 / 1190
Processing Frame 11 / 1190
Processing Frame 12 / 1190
Processing Frame 13 / 1190
Processing Frame 14 / 1190
Processing Frame 15 / 1190
Processing Frame 16 / 1190
Processing Frame 17 / 1190
Processing Frame 18 / 1190
Processing Frame 19 / 1190
Processing Frame 20 / 1190
Processing Frame 21 / 1190
Processing Frame 22 / 1190
Processing Frame 23 / 1190
Processing Frame 24 / 1190
Processing Frame 25 / 1190
Processing Frame 26 / 1190
Processing Frame 27 / 1190
Processing Frame 28 / 1190
Processing Frame 29 / 1190
Processing Frame 30 / 1190
Processing Frame 31 / 1190
Processing Frame 32 / 1190
Processing Frame 33 / 1190
Processing Frame 34 / 1190
Processing Frame 35 / 1190
Processing Frame 36 / 1190
Processing Frame 37 / 1190
Processing

In [7]:
tracks

[array([[ 0., 52.]], dtype=float32),
 array([[ 0., 52.]], dtype=float32),
 array([[31., 52.]], dtype=float32)]

: 